In [2]:
from pathlib import Path

In [3]:
real_input = Path("../input_data/16").read_text()

In [20]:
test_input1 = "8A004A801A8002F478"
test_input2 = "620080001611562C8802118E34"
test_input3 = "C0015000016115A2E0802F182340"
test_input4 = "A0016C880162017C3686B18A3D4780"

In [87]:
def hex_to_bin(hex_string):
    binmap = {
        "0": "0000",
        "1": "0001",
        "2": "0010",
        "3": "0011",
        "4": "0100",
        "5": "0101",
        "6": "0110",
        "7": "0111",
        "8": "1000",
        "9": "1001",
        "A": "1010",
        "B": "1011",
        "C": "1100",
        "D": "1101",
        "E": "1110",
        "F": "1111",
    }
    return "".join([binmap[char] for char in hex_string])


def dec_to_bin(decimal_number, leading_zeros=4):
    if leading_zeros:
        return f"{decimal_number:0{leading_zeros}b}"
    else:
        return f"{decimal_number:#b}"

In [88]:
binary_representation = dec_to_bin(int(real_input, 16), 4)

In [114]:
print_enabled = True


def print_toggle(item):
    if print_enabled:
        print(item)


class Packet:
    subpkg_order = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    def __init__(self, binary_string, pid="0"):
        try:
            print_toggle(f"Creating packet {pid} from {binary_string}")
            self.binary_string = binary_string
            self.i = 0
            self.annotation = ""

            self.pid = pid
            self.subpackets = []

            self.version = int(binary_string[self.i: self.i + 3], 2)
            self.i += 3
            self.annotation += "VVV"

            self.package_type = int(binary_string[self.i: self.i + 3], 2)
            self.i += 3
            self.annotation += "TTT"

            print(f"Header of {self.pid} {self.version=} {self.package_type=}")

            if self.package_type == 4:

                print_toggle("Found a packet of type 4")
                packet_payload = ""
                final_block_found = False

                while not final_block_found:
                    print_toggle(f"Searching block {binary_string[self.i:self.i + 5]}")
                    final_block_found = binary_string[self.i] == "0"
                    self.i += 1

                    packet_payload += binary_string[self.i: self.i + 4]
                    self.i += 4

                    self.annotation += self.subpkg_order[int(len(packet_payload) / 4) - 1] * 5

                self.value = int(packet_payload, 2)

            else:
                length_type_id = binary_string[self.i]
                self.annotation += "I"
                self.i += 1

                if length_type_id == "1":
                    n_subpackets = int(self.binary_string[self.i: self.i + 11], 2)
                    self.annotation += "L" * 11
                    self.i += 11

                    print_toggle(f"Must find {n_subpackets} subpackets.")
                    for _ in range(n_subpackets):
                        subpacket_string = self.binary_string[self.i:]
                        subpacket = Packet(subpacket_string, pid=self.pid + f"_{len(self.subpackets)}")
                        print(f"Added {subpacket} to {self}")
                        self.i += subpacket.i
                        self.annotation += subpacket.annotation
                        self.subpackets.append(subpacket)

                elif length_type_id == "0":
                    n_bits = int(binary_string[self.i: self.i + 15], 2)
                    self.annotation += "L" * 15
                    self.i += 15

                    while n_bits > 0 and "1" in self.binary_string[self.i: self.i + n_bits]:
                        subpacket_string = self.binary_string[self.i: self.i + n_bits]
                        subpacket = Packet(subpacket_string, pid=self.pid + f"_{len(self.subpackets)}")
                        print(f"Added {subpacket} to {self}")
                        n_bits -= subpacket.i
                        self.i += subpacket.i
                        self.annotation += subpacket.annotation
                        self.subpackets.append(subpacket)

        except Exception as e:
            if self.pid == "0":
                print(e)
                print_packets([self])
            raise

    def compute(self):
        """
        Calculations for part 2
        """
        match self.package_type:
            case 0:  # sum
                return sum([p.compute() for p in self.subpackets])
            case 1:  # product
                product = self.subpackets[0].compute()
                for subpacket in self.subpackets[1:]:
                    product *= subpacket.compute()
                return product
            case 2:  # min
                return min([p.compute() for p in self.subpackets])
            case 3:  # max
                return max([p.compute() for p in self.subpackets])
            case 4:
                return self.value
            # TODO 5-7
            case 5:  # >
                first, second  = self.subpackets[0].compute(), self.subpackets[1].compute()
                return int(first > second)
            case 6:  # <
                first, second  = self.subpackets[0].compute(), self.subpackets[1].compute()
                return int(first < second)
            case 7:  # ==
                first, second  = self.subpackets[0].compute(), self.subpackets[1].compute()
                return int(first == second)


    def version_sum(self):
        return self.version + sum([sp.version_sum() for sp in self.subpackets])

    def __repr__(self):
        return f"<Packet version={self.version} package_type={self.package_type} {self.value if self.package_type == 4 else ''} {self.pid=}>"




In [115]:
def print_packets(packets, original_length=None):
    if not original_length:
        original_length = len(packets[0].binary_string)

    for packet in packets:
        indent = original_length - len(packet.binary_string)
        print(" " * indent + str(packet))
        print(" " * indent + packet.binary_string)
        print(" " * indent + packet.annotation)
        print_packets(packet.subpackets, original_length=original_length)


In [116]:
samples = {
    ("110100101111111000101000", 6),
    ("00111000000000000110111101000101001010010001001000000000", 9),
    (hex_to_bin("8A004A801A8002F478"), 16),
    (hex_to_bin("620080001611562C8802118E34"), 12),
    (hex_to_bin("C0015000016115A2E0802F182340"), 23),
    (hex_to_bin("A0016C880162017C3686B18A3D4780"), 31),
}

In [117]:
for n, (bin_string, expected_version_sum) in enumerate(samples):
    print(f"Package {n}")
    p = Packet(bin_string)
    print_packets([p])
    version_sum = p.version_sum()
    print(f"{version_sum=} {expected_version_sum=}")
    print("----")
    if expected_version_sum != version_sum:
        print(f"Wrong version sum {expected_version_sum=} {version_sum=}")



Package 0
Creating packet 0 from 00111000000000000110111101000101001010010001001000000000
Header of 0 self.version=1 self.package_type=6
Creating packet 0_0 from 110100010100101001000100100
Header of 0_0 self.version=6 self.package_type=4
Found a packet of type 4
Searching block 01010
Added <Packet version=6 package_type=4 10 self.pid='0_0'> to <Packet version=1 package_type=6  self.pid='0'>
Creating packet 0_1 from 0101001000100100
Header of 0_1 self.version=2 self.package_type=4
Found a packet of type 4
Searching block 10001
Searching block 00100
Added <Packet version=2 package_type=4 20 self.pid='0_1'> to <Packet version=1 package_type=6  self.pid='0'>
<Packet version=1 package_type=6  self.pid='0'>
00111000000000000110111101000101001010010001001000000000
VVVTTTILLLLLLLLLLLLLLLVVVTTTAAAAAVVVTTTAAAAABBBBB
                             <Packet version=6 package_type=4 10 self.pid='0_0'>
                             110100010100101001000100100
                             VVVTTTAAAAA
  

In [118]:
p = Packet(hex_to_bin(real_input))
p.version_sum()
p.compute()

Creating packet 0 from 10000000010101001111100111001001010111111001110000011100100101110011110100000000000011010000101001111001111101100110001101011001100001100010011100001011000001010100101011101001111011100101000111111000000000000001110100111001010111001100111111100010000100000100001001001001011111100100101000101111011000100000000011100001100000000011101100001100001000001000010001101000001000000000010000110110001100001100000111111000101010000100000000001000011111000110110010001011101100010110100010000000000110000011100101010101010110011010001100001001100101111010100010101110011000000000011001001101000101111001100000000010100100010111001101000000000101100001000000000110001000101111010000011111100011011100001000000000111101000001000110001101001100010111010101000000000011101000100101101100000001101000111010011000000000010110111000000000011110010000000101101001101001100000000001011001000000010100000111101110000000000110001010000000000111101000000001000001111010000000000011110001101000000

834151779165